# Dev notebook for data exploration and model baselines

Preliminary exploration of Russian literature text dataset.

In [434]:
# path to data file
data_path = 'data/tiny-russian-lit/very_clean_tiny_russian_lit.txt'

In [435]:
# read it in for inspection
with open(data_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [436]:
print(f'Length of dataset at {data_path} is {len(text)} characters')

Length of dataset at data/tiny-russian-lit/very_clean_tiny_russian_lit.txt is 34948088 characters


In [437]:
print(f'First 1000 characters of the dataset:\n {text[:1000]}')

First 1000 characters of the dataset:
 Михаил Лермонтов
  

Выхожу один я на дорогу;
Сквозь туман кремнистый путь блестит;
Ночь тиха. Пустыня внемлет богу,
И звезда с звездою говорит.

В небесах торжественно и чудно!
Спит земля в сиянье голубом...
Что же мне так больно и так трудно?
Жду ль чего? жалею ли о чем?

Уж не жду от жизни ничего я,
И не жаль мне прошлого ничуть;
Я ищу свободы и покоя!
Я б хотел забыться и заснуть!

Но не тем холодным сном могилы...
Я б желал навеки так заснуть,
Чтоб в груди дремали жизни силы,
Чтоб, дыша, вздымалась тихо грудь;

Чтоб всю ночь, весь день мой слух лелея,
Про любовь мне сладкий голос пел,
Надо мной чтоб, вечно зеленея,
Темный дуб склонялся и шумел.
Михаил Лермонтов
ВАЛЕРИК
Я к вам пишу случайно; право,
Не знаю как и для чего.
Я потерял уж это право.
И что скажу вам? — ничего!
Что помню вас? — но, боже правый,
Вы это знаете давно;
И вам, конечно, все равно.
И знать вам также нету нужды,
Где я? что я? в какой глуши?
Душою мы друг другу чужды,
Да вр

In [438]:
# find the unique characters that occur in the text
chars = sorted(list(set(text)))
vocab = ''.join(chars)
vocab_size = len(chars)
print(f'Text vocabulary: {vocab}\nVocabulary size: {vocab_size}')

Text vocabulary: 
 !&',-.:;?i ̀́ЁІЉАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёєі–—’
Vocabulary size: 88


Now, we need to be able to tokenize our input - convert raw string text into a sequence of integers according to our vocabulary of possible elements.

For a character-level language model, each character in our vocabulary gets tokenized.

In [439]:
# create a simple character-level tokenizer: a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: convert string to list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: convert list of integers to string

In [440]:
def verify(string):
    print(f"The string '{string}' has the encoding {encode(string)}")
    print(decode(encode(string)) == string)

In [441]:
char = decode([1])
utf8_encoded = char.encode('utf-8')
print(utf8_encoded.hex())

20


In [442]:
verify(' ')
verify('\n')
verify('и')
verify('Мой дядя самых честных правил')

The string ' ' has the encoding [1]
True
The string '
' has the encoding [0]
True
The string 'и' has the encoding [58]
True
The string 'Мой дядя самых честных правил' has the encoding [30, 64, 59, 1, 54, 81, 54, 81, 1, 67, 50, 62, 77, 71, 1, 73, 55, 67, 68, 63, 77, 71, 1, 65, 66, 50, 52, 58, 61]
True


In [443]:
# encode the entire text dataset and store in a tensor
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(f'Input data tensor has shape {data.shape} and type {data.dtype}')
print(f'First 1000 elements of data tensor:\n {data[:1000]}')

Input data tensor has shape torch.Size([34948088]) and type torch.int64
First 1000 elements of data tensor:
 tensor([30, 58, 71, 50, 58, 61,  1, 29, 55, 66, 62, 64, 63, 68, 64, 52,  0,  1,
         1,  0,  0, 20, 77, 71, 64, 56, 69,  1, 64, 54, 58, 63,  1, 81,  1, 63,
        50,  1, 54, 64, 66, 64, 53, 69,  9,  0, 35, 60, 52, 64, 57, 78,  1, 68,
        69, 62, 50, 63,  1, 60, 66, 55, 62, 63, 58, 67, 68, 77, 59,  1, 65, 69,
        68, 78,  1, 51, 61, 55, 67, 68, 58, 68,  9,  0, 31, 64, 73, 78,  1, 68,
        58, 71, 50,  7,  1, 33, 69, 67, 68, 77, 63, 81,  1, 52, 63, 55, 62, 61,
        55, 68,  1, 51, 64, 53, 69,  5,  0, 26,  1, 57, 52, 55, 57, 54, 50,  1,
        67,  1, 57, 52, 55, 57, 54, 64, 80,  1, 53, 64, 52, 64, 66, 58, 68,  7,
         0,  0, 20,  1, 63, 55, 51, 55, 67, 50, 71,  1, 68, 64, 66, 56, 55, 67,
        68, 52, 55, 63, 63, 64,  1, 58,  1, 73, 69, 54, 63, 64,  2,  0, 35, 65,
        58, 68,  1, 57, 55, 62, 61, 81,  1, 52,  1, 67, 58, 81, 63, 78, 55,  1,
        53,

In [444]:
# split data into train and validation sets to test for overfitting
split = 0.8
n = int(split*len(data))
train_data = data[:n]
val_data = data[n:]

Block size, or context length, is the max length of any individual chunk of text that the transformer is trained on. A chunk of text of length `block_size + 1` has `block_size` individual training examples. This also means that the size of the input to the transformer at sampling time will never exceed `block_size`.

In [445]:
block_size = 8
first_block = train_data[:block_size + 1]
print(f'First block of the training data, + 1 character: {first_block}')

First block of the training data, + 1 character: tensor([30, 58, 71, 50, 58, 61,  1, 29, 55])


For a given block of text with length block_size + 1, we will train the transformer on each sequence/target pair from length 1 to block_size (where target is character immediately following the last character in the sequence). This is done so that the transformer is 'used' to predicting the next token given contexts of length as small as 1 and as large as block_size. This is important at sampling time, where the transformer has to begin generating targets from a context of potentially less than block_size.

In [446]:
print(f'Training examples/sequences in first block of data')
for i in range(1, block_size + 1):
    print(f'{i}/{block_size}: When input is, {first_block[:i]} target is {first_block[i]}')

Training examples/sequences in first block of data
1/8: When input is, tensor([30]) target is 58
2/8: When input is, tensor([30, 58]) target is 71
3/8: When input is, tensor([30, 58, 71]) target is 50
4/8: When input is, tensor([30, 58, 71, 50]) target is 58
5/8: When input is, tensor([30, 58, 71, 50, 58]) target is 61
6/8: When input is, tensor([30, 58, 71, 50, 58, 61]) target is 1
7/8: When input is, tensor([30, 58, 71, 50, 58, 61,  1]) target is 29
8/8: When input is, tensor([30, 58, 71, 50, 58, 61,  1, 29]) target is 55


In [447]:
torch.manual_seed(3)
batch_size = 4  # the number of independent sequences that we will process in parallel
block_size = 8  # maximum context length for predictions

def get_batch(split):
    # generate a batch of data consisting of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # generate batch_size random offsets in the interval [0, len(data) - batch_size)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-' * 10)

for b in range(batch_size): # batch dimension
    print(f'Batch {b + 1}/{batch_size}')
    for t in range(block_size): # time/position dimension
        context = xb[b, : t+1]
        target = yb[b, t]
        print(f'When input is {context.tolist()}, target is {target}')

inputs:
torch.Size([4, 8])
tensor([[65, 66, 64, 81, 52, 58, 61, 64],
        [65, 64, 54, 63, 58, 62, 50, 68],
        [ 1, 68, 64, 53, 64,  5,  1, 73],
        [57, 63, 50, 73, 55, 63, 78, 55]])
targets:
torch.Size([4, 8])
tensor([[66, 64, 81, 52, 58, 61, 64, 67],
        [64, 54, 63, 58, 62, 50, 68, 78],
        [68, 64, 53, 64,  5,  1, 73, 68],
        [63, 50, 73, 55, 63, 78, 55,  0]])
----------
Batch 1/4
When input is [65], target is 66
When input is [65, 66], target is 64
When input is [65, 66, 64], target is 81
When input is [65, 66, 64, 81], target is 52
When input is [65, 66, 64, 81, 52], target is 58
When input is [65, 66, 64, 81, 52, 58], target is 61
When input is [65, 66, 64, 81, 52, 58, 61], target is 64
When input is [65, 66, 64, 81, 52, 58, 61, 64], target is 67
Batch 2/4
When input is [65], target is 64
When input is [65, 64], target is 54
When input is [65, 64, 54], target is 63
When input is [65, 64, 54, 63], target is 58
When input is [65, 64, 54, 63, 58], target i

Probably the simplest language model is a bi-gram with character-based tokens. Given a single character, it predicts the next character in the sequence. I now implement a bi-gram as a baseline for our Russian text generation task.

In [448]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off the logits (input to softmax) for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers (B = # batches, T = # timesteps/block size)
        # we are essentially predicting the next character based on the embedding of a single token
        logits = self.token_embedding_table(idx)  # (B, T, C) : batch, time, channels
        
        if targets is None:
            loss = None
        else:
            # reshape logits since cross_entropy expects (B, C, T) inputs
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)  # equivalently, targets.view(-1)

            # negative log likelihood loss - calculates quality of our logits with respect to the true targets
            # a 'good' logit will have a high value in the target dimension and low values in other dimensions
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        # the bigram only uses the last char as the context
        # we pass in the full context here as practice for generation using transformer
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)  # calls the forward function
            # retrieve only final timestep
            logits = logits[:, -1, :] # (B, T, C) -> (B, C)
            # apply softmax to get probability distribution
            dist = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(dist, num_samples=1) # (B, 1)
            # append new sample to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx


In [449]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)  # 4 batches, 8 timesteps, vocab_size channels
print(loss)

torch.Size([32, 88])
tensor(5.1279, grad_fn=<NllLossBackward0>)


In [450]:
torch.manual_seed(3)

def sample(context, new_tokens=100):
    print(f'Context: {decode(context[0].tolist())}')
    sample = model.generate(context, new_tokens)
    text = decode(sample[0].tolist())
    print(f'Sample: {text}')


# as the model's starting context for sampling, let's provide a newline character
blank_context = torch.tensor([encode('\n')])
sample(blank_context, 250)

Context: 

Sample: 
ОГфРиРОЉЪЕп&&лыЫЛТ 'эятІМЦИсы&.ІМЭОнПЯАЪЖоє;ащМнюБи;
ксрршА.ЪЗмн'i—йОщУфйдыцЗыiйсшЧЦЬяяИ—н:зфВУП–?—Љ
–
'Псiя&.Кєхюкг;тІВтбНЬКщыЗСИ.ЩЖр—єКаЉШЭ:Ді сДХРиК?вГрф̀-щЛЛЛН́ЙiДІфгуГАп—р;Ъ,эБгь́Фяьк ЖД–нЮкхКдл'ууХе
И–м—ЖчнСКюОЫєжє̀́іРПЪР&д—эгНіясяХгБ!Л,шЯОе'ом


The above sampled text is gibberish. Let's train the model so it can produce something that looks more reasonable.

In [451]:
# typical lr setting is 3e-4, but for small models we can use a much higher lr
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [452]:
batch_size = 32
num_steps = 10000
for step in range(num_steps):
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if step == 0 or step == num_steps - 1:
        print(f'Step {step + 1}/{num_steps}: loss={loss.item()}')

Step 1/10000: loss=4.836124897003174
Step 10000/10000: loss=2.6156206130981445


After optimization, let's see if we can sample something more reasonable.

In [457]:
sample(blank_context, 250)

Context: 

Sample: 
Во нодогого сло пряЮЛФа вахаят врев вучт промадоскомукомы, вето о, оды Налах. м ецм я ди, быпотел тьсяда че сть резетмя ни прброрарбыт, ужном.  были, умодх,
— бла:
РЩвшазмав е? дне- м вь; бы нстабый сямов х, им — петоЙКозднь, вл дреглтимика.. нумубни
